# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [7]:
browser = webdriver.Chrome(executable_path='C:/Users/JoeZhong/.spyder-py3/chromedriver')
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁


#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("return document.body.scrollHeight;")
while True:
    time.sleep(SCROLL_PAUSE_TIME)
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    height = browser.execute_script("return document.body.scrollHeight;")
    if last_height == height:
        break
    last_height = height

6804
8674
8674
8724
8724
11108
11108
13506
13506
15879
15879
18153
18153
20498
20498
22741
22741
24158
24158
24158


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [8]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [9]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 財訊／中國廠賣不賣？　可成董座洪水樹陷入兩難

文／林宏達
過去幾個月，蘋果供應鏈在蘋果指揮下，形成「2套系統」的狀況愈來愈明顯，繼緯創把中國廠賣給立訊之後，現在可成的動向最受關注。
在iPhone供應鏈裡，組裝的獲利不高，真正賺錢的是零組件，過去iPhone機殼的主力廠商是鴻準和可成，蘋果今年大力要促成機殼廠和組裝廠「聯姻」。可成董事長洪水樹在今年股東會上也說：「有興趣找我們談合作，都會談，目前為止沒有跟任何一家公司有具體進展。」
不過，市場人士透露，立訊和藍思都有意整併可成中國廠，但可成開出400億元人民幣（約合台幣1716億元）的高價，讓有意上門的買家碰了個軟釘子；畢竟，光是今年第1季，可成的帳上現金高達1178億元，並不缺錢。
延伸閱讀：
疫情催化供應鏈大洗牌 蘋果缺貨潮湧現，這些台廠可望獲益
立訊牽動蘋果供應鏈變化 謝金河：下個戰場在「這裡」
----------------------------------------------------------------------

[1] 財訊／劉德音5年前開始悄悄布局　台積電挑戰500元的祕密武器

文／財訊新聞中心
近來投資市場最重要的議題，幾乎全部與台積電有關。回顧20年前，英特爾市值是台積電的6.2倍；如今，英特爾雖仍是半導體巨人，但台積電市值卻已經超越英特爾。台積電是怎麼做到的？最新一期《財訊》雙週刊仔細盤點台積電整個合縱連橫的過程，發現台積電早就默默布局許久。董事長劉德音到底做了什麼關鍵決定？同時解析台積電股價上攻500元的祕密武器。
台積電第1個10年，大客戶都來自PC產業；第2個10年，靠手機登上兆元；如今，台積電在高性能晶片製造上，實力超越英特爾，股價離500元並不遠。其實這一切，早在5年前時，劉德音就已開始悄悄部署。
7月底，一場遠在美國的法說會，宣告台積電在高速運算時代的大機會正式到來。這一天，英特爾執行長史旺（Bob Swan）透露兩個重要的訊息：「我們7奈米的處理器生產時程，會比原先預期的晚6個月。」他接著解釋，為了維持「英特爾產品在市場上的領導地位，將會在自家晶圓廠之外，採用其他晶圓代工廠產能生產先進產品」。法說會後，英特爾隔

▲「花見」集合住宅由雄興建設投資興建，成為北市指標性的「危老重建工程」第一案。
圖、文／雄興建設提供
台北市政府為加速都市更新，積極推動「危老條例」重建工程，首宗「危老重建工程」的基地就是位於台北市長安東路一段40號的「花見」。該集合住宅由雄興建設投資興建，該案受市長柯文哲關注，成為北市指標性的「危老重建工程」第一案。
「花見」案座落於華山藝文特區，基地約100坪，規劃地上10層及地下3層，基地雖小然而透過本次重新規劃後除了建物四周退縮建築獲得更好的採光通風外，另外也規劃符合耐震設計、綠建築設計、智慧建築設計、無障礙設計等標章，並依法申請容積獎勵達最高上限法定容積40%，除了可提升居住安全外，也可帶動周邊區域發展，可堪稱是全台北小型基地之模範。
雄興建設董事長蔡秋鑾在2017年買下土地後，只花10個月就跑完所有流程，順利取得建照，去年初建案對外預售，3個月就完銷，每坪成交均價近90萬元，在當時低迷的房地產景氣中一枝獨秀。雄興建設本著遵照市府推動都市更新計畫的宗旨，對「危老重建」工程努力不懈，並將於下半年陸續在信義計畫區推出危老重建第二案「興百世」及後續大安區第三案等等。希望藉著這些危老專案延續雄興建建設的精神，將綠建築、耐震、智慧、無障礙設計融入現代建築中。
----------------------------------------------------------------------

[13] 新台幣收盤貶值1.1分　收在29.514元

▲新台幣收盤收29.514元。（圖／記者湯興漢攝）
記者陳心怡／台北報導
台北外匯市場新台幣兌美元匯率今（7）日收盤收29.514元，貶值1.1分，成交金額8.37億美元。
新台幣今日以29.510元兌1美元，貶值0.7分開出，開出之後隨即反轉向上，一度觸及29.361元價位，強彈1.42角，午盤後升幅收斂，收盤收29.514元，貶值1.1分。
新台幣兌美元盤中最高29.361元，最低價則是29.515元兌1美元。
----------------------------------------------------------------------

[14] 旭隼攻漲停突破千元大關　台股第六大千金出線

▲台股六大千金出線。（資料圖／實習記者方敬攝）
記者姚惠茹／台北報導
不斷電系統（UPS）廠旭隼（640

▲華南銀行總部。（圖／記者吳靜君攝）
記者吳靜君／台北報導
國內新冠肺炎（COVID-19）疫情趨緩，走過疫情，一起運動提高身體免疫力吧！教育部體育署投下高達20億元發放400萬份500元的動滋券，8到12月可於運動場館、觀賞運動賽事、購買運動用品等相關運動產業抵用。
為響應政府「動滋券」挺運動，華南銀行信用卡舉辦「1788一起運動吧」活動，即日起至10月底止，至全台指定運動用品專賣店，如摩曼頓、迪卡儂、尚智運動世界、萬岳運動休閒館、哈林運動世界、墾趣生活、山頂鳥、歐都納，刷華南銀行信用卡單筆消費滿1,788元，登錄即送178元刷卡金，回饋率高達10%，加計華南信用卡原有回饋如享利樂活COMBO卡新戶，回饋率達11.3%，若再搭配動滋券使用，回饋率上看30%。
----------------------------------------------------------------------

[25] 北富銀總座爆出授信弊案　金管會從2方向來查不排除約談說明

▲金管會。（圖／記者湯興漢攝）
記者紀佳妘／台北報導
台北富邦銀行今（7）日遭爆出易恒健康授信弊案，指總經理程耀輝涉及忽略銀行內部對該案的風險通報，要求盡速核貸3000萬美元的貸款，同時疑似以程耀輝作為最終受益人等消息，金管會對此表示，會從2方面請北富銀說明，包括易恒健康授信案、涉及「匯款」部分，因此會待銀行提供資料後再來判斷。
金管會銀行局長莊琇媛表示，昨已接到消息，目前請北富銀提供易恒健康授信情況、匯款等相關資料，請該行盡快繳交資料。
▲北富銀總經理程耀輝。（圖／資料照）
針對北富銀總經理程耀輝涉及弊案一事，莊琇媛指出，「我們先請銀行提供資料後，視情況採取後續行動」。至於後續是否約談總經理？莊琇媛說，這也要看北富銀報來的資料後，視情況決定是否請總經理來說明。
目前情節是否涉及不當授信，莊琇媛指出，目前媒體揭露的訊息與內部得到的消息一樣，會從2方面請北富銀說明，首先是易恒健康授信案，了解整個授信過程是否符合內部規範和法規，其次為一筆涉及「匯款」部分，因此會待銀行提供資料後再來判斷。
外界關注此案是否「冰山一角」，後續銀行局是否擴大徹查？莊琇媛表示，針對易恒健康授信案或相關案件來查看看，再判斷是否有必要進一步擴大查核。
----------------------------------------

記者王佩翊／綜合報導
受到新冠肺炎影響，為減少傳染風險，各國紛紛在疫情最盛期要求民眾減少外出，許多公司也發展出在家工作的新形態，進而促進「宅經濟」發展。由任天堂發行的Switch因此再度爆紅，甚至數度斷貨，任天堂6日公布第一季（4月至6月）財報，其中，營業額與前一年同期相比增長至2.1倍，淨利潤更是一舉飆升至6.4倍，可以說是疫情下的最大受惠者。
根據《共同社》報導，日本任天堂公司6日發表2020年度Q1財報，根據財報顯示，任天堂本季度淨利潤高達1064億日幣（約新台幣295億元），是為2019年同期的6.4倍。此外，合併營收為3581億日幣（約新台幣995億元），為前一年同期的2.1倍。據悉，利潤較高的遊戲軟體下載次數增高，對淨利潤帶來極大貢獻。
在Q1中，Switch主機共銷售了568萬台（包含一般版主機與Switch Lite），增加至2.7倍。而Switch平台遊戲軟體銷量暴增至5043萬套，是2019年同期的2.2倍，其中光是爆紅的「集合啦！動物森友會」就狂賣了1063萬套。
任天堂表示，受到疫情影響，許多工廠被迫停工，導致Switch主機的生產一度停擺，不過如今生產線已大致恢復正常，雖然部分地區的主機「仍在缺貨中」。
即使第一季財報表現亮眼，但任天堂仍決定維持2020年度的財測預估，合併營收欲將減少8.3%至1.2兆日幣（約新台幣3336億元），淨利潤減少22.7%至2000億日幣（約新台幣556億元）。任天堂目標今年度的Switch主機全球銷量為1900萬台。
----------------------------------------------------------------------

[39] 找出下個蘋果、亞馬遜！　謝金河曝「4潛力股」有亮點：可挖掘一下

▲財信傳媒董事長謝金河。（圖／ETtoday資料照）
記者林妤柔／綜合報導
蘋果、亞馬遜經營成功，股價亦水漲船高。財信傳媒董事長在個人臉書分享「大金剛法則的投資啓示」，認為股票高昂的企業，未來要創造成高投資報酬率，恐怕很困難。他建議，如果要選未來的大金剛，可從今年嶄露頭角的企業找起。他也點出4家表現不錯的企業，認為可以好好挖掘一下。
謝金河在個人臉書貼文中分享台灣一本暢銷書，叫做「大金剛法則」（The Gorilla Game），作者是Geoffrey A. Moore，用猴子、

新聞節目中心／綜合報導
大同公司經營權爭奪戰持續延燒！6月30日公司派上演剔除53％市場派表決權、三百黑衣人圍場的戲碼，轟動社會。市場派為了挽回局面，近期爭取向主管機關申請自行召開股東臨時會，但這場股東臨時會能開成嗎？由資深司法記者蘇位榮主持的《行動法庭》，昨（6）日邀請國立台北大學法律學系教授陳彥良、洪國誌律師，探討《公司法》第173條第4項能否作為突破公司派董事會封殺的出口？
▲ 洪國誌律師分析，由大同董事會召開股東臨時會將面對的法律爭議。（圖／行動法庭提供）
憂大同董事會再次違法　市場派盼自行召集股臨會
6月30日，由公司派主導的股東會（簡稱630股東會），因剔除53％以上股東表決權、拒發股東選票、數百黑衣人圍場，衝擊股東權益，引發社會議論。後來公司派依630股東會決議申請變更董事登記，於7月9日遭經濟部駁回。而金管會也以特別背信罪，將林郭文豔移送檢調單位。
因此律師洪國誌認為，市場派可行使《公司法》第173條第4項的權利，由3%的股東報請主管機關許可，自行召集股東臨時會。
▲ 洪國誌律師認為，由大同公司董事會召開股東會有程序爭議。（圖／行動法庭提供）
學者：《公司法》173條4項仍有解釋空間
若市場派要主張《公司法》第173條第4項，依法須先符合「董事因股份轉讓或其他理由，致董事會不為召集或不能召集股東會時」的要件。
台北大學法律學系教授陳彥良表示，綜合過去經濟部函釋，以及最高行政法院、行政法院判決等歷史脈絡，過去實務見解對第173條第4項的「其他理由」採限縮解釋。
所謂「其他理由」，實務見解認為是指，董事全體辭職，或全體出去玩，坐飛機全部摔死；或者全部被定暫時狀態假處分，被禁止執行職務，「致」董事會不為或不能召集股東會等情境。
依據陳彥良教授教書多年的經驗，以及他對行政和司法體系的了解，他認為公部門會採取相對保守的看法。對於市場派用《公司法》第173條第4項，他認為勇氣可嘉，可以嘗試，但如果要「突破僵化的主管機關跟法院」，「人人都有希望，但機會恐怕不是那麼大。」
「要法院勇敢，主管機關才會更勇敢。」陳彥良教授說道。這時候就要靠「投保中心」，最慘的是大同的員工，一個個被裁員，連生計都沒有！
▲ 國立台北大學法律學系教授陳彥良認為，依《公司法》第173條第4項召開股東會「還是有一點點的空間」。（圖／行動法庭提供）
律師：主管機關可做「實質審查」
講到最後，洪國

▲台翰越南廠。（圖／台翰提供）
記者姚惠茹／台北報導
台翰精密（1336）今（6）日公布7月合併營收1.78億元，月增19.17%，創今年來單月新高，主要是受到疫情趨緩，東南亞各地區政府以漸進式復甦經濟活動，累計今年前7月合併營收9.9億元，年減26.22%。台翰表示，最壞的情況已過，下半年業績可望比上半年好。
台翰表示，上半年營運受到新冠肺炎疫情影響，但菲律賓廠區自5月16日復工後，可見主要客戶的拉貨力道有逐步恢復的跡象，公司與客戶積極拉高產能稼動率以因應整體印表機/事務機的需求，目前產能稼動率已達50%水準，而越南廠的產能利用率也已達70%。
台翰指出，雖然疫情短期影響訂單遞延出貨，但目前已與主要客戶積極針對新機種進行認證，若順利按照時程進度，有機會在今年第3季試量產，未來隨著客戶需求增加，將有助於整體產能規模放大，並強調最壞的情況已過，下半年本業營運有望恢復過往成長表現，可望比上半年好。
台翰說明，目前越南廠區佔整體營收55%、菲律賓廠區約佔35%、東莞廠約佔10%，而且東莞廠將會持續縮編，預計明年僅剩60人左右，作為模具開發設計使用，主要是因為大陸生產成本越來越高，但今年由於有新模具開發，所以第3季的比重則是以東莞廠為主。
根據中國海關總署公布今年上半年最新貿易統計數據顯示，大陸與所有國家及地區的貿易額表現來看，由於大陸企業將生產線轉移至東盟國家，創造大陸與東盟(ASEAN)的進出口貿易額首次升至第一，而且近期歐美、日韓等各國也積極祭出補貼計畫，鼓勵該國企業遷回該國家或東南亞區藉以分散產能布局、確保產品與材料的順利供應，有利於東南亞產業鏈群聚發展。
展望下半年營運，台翰表示，看好東南亞兩個生產基地產能規模放大，貢獻整體營運穩健基礎，雖然東南亞地區政府仍實施防疫措施，但仍以最低限度生產方式發揮最大量產價值，滿足主要客戶的拉貨需求，有助於下半年營運保持良好的成長動能。
台翰強調，公司持續強化模具設計、塑膠射出的量產技術，將憑藉著深耕東南亞市場超過15年，積極爭取更多與不同產業客戶合作的可能性，藉以保持高度的訂單能見度，並對整體營運帶來正面的挹注。
此外，台翰閒置資產處分利益2.13億元已在今年第2季入帳，同時也在上半年完成庫藏股買回4000張，目前股本為7.71億元，未來台翰仍將持續落實充實營運資金、強化財務結構、提高財務運用靈活度及提升股東權益，致力將不確定因

圖文／鏡週刊
今年33歲的韋禮安，睽違4年推出第5張專輯《Sounds of My Life》，紀錄生活中所在乎的聲音，是一張充滿愛與感謝的作品。創作音樂感性十足的他，3年前就開始為退休鋪路，透過大量閱讀奠定長期投資的道路，理性分析ETF最適合自己。
從出道專輯裡的〈慢慢等〉、〈有沒有〉，到電視劇片頭曲〈還是會〉、〈女孩〉，韋禮安的作品傳唱度極高，深受大眾歡迎。但好好的歌手怎麼會開始投資呢？原來是出於對未來的不安感，「2017年快邁入30歲時，我意識到演藝人員工作不穩定，退休金得做好規劃。」韋禮安說。
「扣掉每月生活費，幾乎80%收入都放在經營公司與ETF投資上。」他坦言，當歌手又開公司，職涯上已承受相當高的風險，因此投資上應選擇相對穩健的ETF。
其實早在13年前，剛上大學的韋禮安就在父親的規劃下，開始定期定額買基金，直到現在都還持續扣款。出身書香世家的他，父母皆是外文系教授，「家人在投資理財上很保守，當初也是先跟爸爸討論，才慢慢開始投資海外ETF。」韋禮安解釋。
「投資多年的基金累積報酬雖可觀，但換算成年報酬也只比定存好一點，反而ETF績效更好。」韋禮安表示，目前投資部位有4分之3放在ETF上，分別是：VTI（範圍涵蓋美股大中小型公司，等於買下美國整體市場表現）、VEU（投資美國以外的已開發國家與新興市場，共持有45個國家）、BND（主要投資美國公債、投資級公司債，以及美元計價的國際債券等）。
「VTI加VEU的組合，等於投資全世界股市，BND則是美國整體債券市場。」韋禮安分析。而現年33歲他，股債比約7比3，投資3年下來平均年賺約7%。
事實上，韋禮安買的3檔ETF都是美國先鋒集團（Vanguard）發行的產品，為何獨鍾此集團？韋禮安解釋，先鋒的創辦人約翰‧伯格（John Bogle）是指數基金教父，集團長年深耕ETF商品，且內扣費用率最低。以他投資的ETF為例，VTI總管理費用0.03%、VEU為0.08%、BND為0.035%，「比起共同基金動不動就3%的內扣費用，投資成本差了100倍。」。
----------------------------------------------------------------------

[72] 張張拿1700元！兆豐金「填息不用一年」心動了　網看傻：誰要定存

▲原PO好奇，有閒錢卻不買股票是不是很傻？（圖

文／洪寶山
七月三十一日的台股最後一盤意外的跌了62.23點，跌得有點令人丈二金剛摸不著頭腦，晚上六點中央社發布台灣第二季經濟成長率為負0.73%，就揭開謎底了，經濟衰退的主要原因是受疫情影響實施邊境管制，來台人數減少99.57%，旅行收入、航空、旅宿、餐飲等明顯受累。
在COVID-19新冠病毒無差別的攻擊下，各國都同樣實施邊境管制，因此這場疫情正好檢視全球各國自身經濟結構的健全性，過去三十年在中國改革開放的人口紅利，不管是基礎建設的原物料需求，或是提升生活品質的民生消費需求，為2000年網路泡沫之後的全球經濟成長提供了巨大貢獻。
外銷出口過度依賴中國　邊境管制加速GDP衰退
但福兮禍兮，如今全球自由化貿易走到盡頭，取而代之的是自由貿易下的新保護主義，凡是外銷出口過度依賴中國的國家，在剛公布的第二季GDP成長率，都遭到明顯的衰退，例如新加坡負12.6%、香港為負9%、美國負9.5%，相對於陸客觀光早在四年前就已經趨近於零，台灣第二季GDP僅負0.73%，受到的影響相對輕。
換言之，在邊境管制無法解除，過度依賴中國觀光客的精品業已經首當其衝，LVMH上半年經常性業務利潤為16.7億歐元(約19.6億美元)，差過市場預期的23.2億歐元，由此可見，邊境管制一日未除，全球經濟要想復甦到疫情發生之前，恐怕只能紙上談兵。
口罩可能一輩子都摘不掉　歐美還在糾結
樂觀者指望疫苗問世後，世界就會快速恢復到今年初的榮景，就連台灣的主計處在第二季GDP的新聞稿中也表示，疫情的因素是短期現象，這就可以解釋為什麼經濟數據如此之差，但美股與台股卻能頻創新高，因為有疫苗可期盼。
在媒體的報導，則是說疫苗最快秋季到年底問世，所以都抱著再忍一下就過去了的心態。真好，人有希望的活著是件好事，但事實往往是殘酷的，打個比方，就算問世的疫苗有效，初期的產能能夠提供多少國家施打？哪些人優先施打？就算大量生產，有可能讓全球人口同時完成疫苗接種嗎？且先不論抗體能在人體內持續多久，假如境外的疫情因第三世界國家居高不下，邊境管制能解除嗎？相信很多人大概沒有意識到這輩子可能口罩都摘不掉吧。
而且戴口罩這件事，看似防疫措施，應該無可厚非，但對於歐美國家而言，口罩牽扯了意識形態的糾纏，美國有三Ｋ黨的陰影，德國現在傳出兩萬人上街頭抗議「口罩奴化了我們」，所以邊境管制真的只是短期現象嗎？
中國用內需消費力　經濟制裁「五眼聯

文／陳雅玲
眼看近年國際大廠相繼來台投資，今年4月行政院再祭出政策利多，拍板通過7年砸百億，吸引國際大廠來台落地，藉此推動國內產業轉型升級，全力朝向全球高科技研發中心發展。
然而，當政府帶頭促進產業升級的同時，舊廠房跟著升級也成了不可抵擋的趨勢；再加上台商加速回流，急尋工業區土地，廠房供不應求，造就商辦比住宅搶手，而廠辦比商辦更炙手可熱的現象。
富創得科技公司董事長吳明發，早在2年前就看準這波趨勢，擁有坐落於林口工業園區的廠房，占地2800坪，是林口工二工業區面積最廣的，也是區內首開先例通過的危老重建案。富創得科技預計近期拆除舊廠房，2、3年後將有一棟外觀新穎，規格比照無塵室等級的地下4樓、12層高的立體化科技廠房，企圖打造新創資源共享的生態圈，吸引高科技產業進駐。
新創廠辦　具政策優勢
談到當初為何決定重建舊廠房？吳明發精神抖擻地說，「看到林口是要改變的時候了。」他口中說的改變，正是經濟部2017年啟動「林口新創園」計畫，以林口世大運選手村作為據點，招攬國內外加速器與新創業者進駐，有意打造為亞洲最大的AI聚落。
另一方面，林口正是位於「新創走廊」的中心點，占有絕佳的地理位置，而所謂的新創走廊，主要鏈結桃園市的亞洲．矽谷計畫、新竹的科學園區、台北小巨蛋的國際科技創業基地，也是政府發展國際新創聚落的終極目標。
跟著政策準不會錯，這是吳明發創業30、40年來的商業嗅覺。對這一波台商回流，他觀察到產業發展的脈絡，「大家都知道台商回流，但也要知道一件事，台商回來，並不會投入大量生產的勞力密集型產業，而是會選擇在台灣設立研發中心，把研發技術留在台灣，不然在別人家屋簷下做研發，還沒量產前，技術都被學走了！」
延伸閱讀：
電子代工廠跨足建築業 唯謙50年老廠大變身，馬上增值5倍！	
各路人馬搶插旗、全台近30案審查過關 廠房立體化，千億商機大爆發
----------------------------------------------------------------------

[88] 宏達電7月營收2.84億元　月減56%創20年新低

▲HTC新店總部。（圖／記者湯興漢攝）
記者姚惠茹／台北報導
宏達電（2498）今（6）日公布7月自結合併營收2.84億元，月減56%，年減35％，創20年新低，主要是因為5G新機要等到8月上市，所以7月缺少營運動能；累積今年前7月